In [0]:
import pandas as pd
import numpy as pd
import tensorflow
import cv2 

In [ ]:
TRAIN_DIR = '../data/train'
TEST_DIR = '../data/test'
IMG_SIZE = 224

In [ ]:
def process_data(path_dir): 
    data = []
    img_nums = []
    i = 0
    for img in os.listdir(path_dir)[:50]: 
        path = os.path.join(path_dir, img) 
        img_num = img.split('.')[0] 
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) 
        data.append(np.array(img))
        img_nums.append(img_num)

        if i == 50:
            break
          

    return data, img_num

In [ ]:
x_train, img_nums = process_data(TRAIN_DIR)

In [ ]:
x_train.shape

In [ ]:
labels = pd.read_csv('../data/train_labels.csv')

In [ ]:
y_train = []
for n in img_nums:
    y_train.append(int(labels[labels['name'==n]['invasive']])) #.iloc[0]